# ⚡ DSPy ReAct Agents - Otimização Avançada (Abordagem Hands-On)

**Versão:** 1.0 - Advanced Hands-On  
**Nível:** Intermediário/Avançado  
**Tempo estimado:** 30-45 minutos  
**Abordagem:** Otimizar agora → Ver resultados → Entender técnica

---

## 📋 Sobre este Notebook

Este notebook oferece uma **abordagem prática e direta** para otimizar agentes ReAct.

Você vai:
- ⚡ **Otimizar** seu agente em minutos
- 📊 Ver **melhorias dramáticas** imediatamente
- 🔍 Entender **o que aconteceu** depois dos resultados
- 🧪 **Experimentar** com diferentes técnicas
- 🚀 **Deployar** em produção rapidamente

**Filosofia:** Wow effect primeiro → Entender depois → Experimentar livremente

### 🎯 Pré-requisitos

**Essencial:**
- Um agente ReAct funcional (dos notebooks básicos)
- Python intermediário
- Vontade de experimentar!

### 📚 Navegação entre Notebooks

**Série DSPy ReAct Agents:**
1. [Fundamentos (Linear)](dspy_agents_basic_linear_final.ipynb) - Conceitos básicos
2. [Fundamentos (Hands-On)](dspy_agents_basic_handson_final.ipynb) - Prática básica
3. [Otimização Avançada (Linear)](dspy_agents_advanced_linear_final.ipynb) - Mesma otimização, teoria primeiro
4. **→ Você está aqui:** Otimização Avançada (Hands-On)

---

Based on: https://dspy.ai/tutorials/customer_service_agent/

## ⚡ Vamos Otimizar AGORA!

**Filosofia:** Ver resultados impressionantes primeiro, entender depois!

Execute as células abaixo e prepare-se para ver seu agente melhorar significativamente.


## Setup and Imports

In [85]:
import dspy
import os
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
import json
import uuid
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## Enable Tracing for DSPy

In [87]:
from openinference.instrumentation.dspy import DSPyInstrumentor
DSPyInstrumentor().instrument()

Attempting to instrument while already instrumented


## Data Models

Define Pydantic models for the data structures we'll use.

In [88]:
class Date(BaseModel):
    year: int
    month: int
    day: int
    
    def __str__(self):
        return f"{self.year:04d}-{self.month:02d}-{self.day:02d}"

class UserProfile(BaseModel):
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None

class Flight(BaseModel):
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    
class Itinerary(BaseModel):
    itinerary_id: str
    user_id: str
    flights: List[Flight]
    total_price: float
    booking_date: str
    status: str  # "confirmed", "cancelled", "pending"

class Ticket(BaseModel):
    ticket_id: str
    user_id: str
    itinerary_id: str
    confirmation_number: str
    issue_description: Optional[str] = None
    status: str  # "active", "resolved", "pending"

## Mock Database Setup

Create dummy databases for users, flights, itineraries, and tickets.

In [89]:
# Mock databases
users_db = {
    "Adam": UserProfile(
        name="Adam",
        user_id="user_001",
        email="adam@example.com",
        phone="+1-555-0101",
        frequent_flyer_number="FF12345"
    ),
    "Sarah": UserProfile(
        name="Sarah",
        user_id="user_002",
        email="sarah@example.com",
        phone="+1-555-0102"
    )
}

flights_db = {
    "SFO-JFK": [
        Flight(
            flight_id="f001",
            flight_number="AA101",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="08:00",
            arrival_time="16:30",
            duration_minutes=330,
            price=450.00,
            available_seats=15
        ),
        Flight(
            flight_id="f002",
            flight_number="UA205",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="14:00",
            arrival_time="22:45",
            duration_minutes=345,
            price=380.00,
            available_seats=8
        )
    ],
    "JFK-LAX": [
        Flight(
            flight_id="f003",
            flight_number="DL302",
            departure_airport="JFK",
            arrival_airport="LAX",
            departure_time="10:00",
            arrival_time="13:30",
            duration_minutes=390,
            price=520.00,
            available_seats=12
        )
    ]
}

itineraries_db = {}
tickets_db = {}

## Tool Functions

Define the tools that our agent will use to interact with the airline system.

In [90]:
def fetch_flight_info(departure: str, arrival: str, date: str) -> str:
    """
    Fetch available flights for a specific route and date.
    
    Args:
        departure: Departure airport code (e.g., 'SFO')
        arrival: Arrival airport code (e.g., 'JFK')
        date: Flight date in YYYY-MM-DD format
    
    Returns:
        JSON string with available flights
    """
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights found for route {route} on {date}"})
    
    flights_data = [flight.model_dump() for flight in flights]
    return json.dumps({"flights": flights_data, "count": len(flights_data)})

def pick_flight(departure: str, arrival: str, preference: str = "duration") -> str:
    """
    Pick the best flight based on user preference.
    
    Args:
        departure: Departure airport code
        arrival: Arrival airport code
        preference: 'duration' for shortest flight, 'price' for cheapest
    
    Returns:
        JSON string with selected flight
    """
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights available for route {route}"})
    
    if preference == "duration":
        best_flight = min(flights, key=lambda f: f.duration_minutes)
    else:  # price
        best_flight = min(flights, key=lambda f: f.price)
    
    return json.dumps({"selected_flight": best_flight.model_dump(), "reason": f"Best {preference}"})

def get_user_info(name: str) -> str:
    """
    Retrieve user profile information.
    
    Args:
        name: User's name
    
    Returns:
        JSON string with user profile
    """
    user = users_db.get(name)
    if not user:
        return json.dumps({"error": f"User {name} not found"})
    
    return json.dumps({"user": user.model_dump()})

def book_flight(user_name: str, flight_id: str, date: str) -> str:
    """
    Book a flight for a user.
    
    Args:
        user_name: Name of the user
        flight_id: ID of the flight to book
        date: Travel date
    
    Returns:
        JSON string with booking confirmation
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    # Find the flight
    flight = None
    for route_flights in flights_db.values():
        for f in route_flights:
            if f.flight_id == flight_id:
                flight = f
                break
        if flight:
            break
    
    if not flight:
        return json.dumps({"error": f"Flight {flight_id} not found"})
    
    if flight.available_seats <= 0:
        return json.dumps({"error": "No available seats"})
    
    # Create itinerary
    itinerary_id = str(uuid.uuid4())
    confirmation_number = f"CONF{uuid.uuid4().hex[:8].upper()}"
    
    itinerary = Itinerary(
        itinerary_id=itinerary_id,
        user_id=user.user_id,
        flights=[flight],
        total_price=flight.price,
        booking_date=datetime.now().strftime("%Y-%m-%d"),
        status="confirmed"
    )
    
    itineraries_db[itinerary_id] = itinerary
    
    # Update available seats
    flight.available_seats -= 1
    
    return json.dumps({
        "success": True,
        "confirmation_number": confirmation_number,
        "itinerary_id": itinerary_id,
        "flight": flight.model_dump(),
        "total_price": flight.price,
        "message": f"Flight {flight.flight_number} booked successfully for {user_name}"
    })

def cancel_itinerary(itinerary_id: str) -> str:
    """
    Cancel an existing itinerary.
    
    Args:
        itinerary_id: ID of the itinerary to cancel
    
    Returns:
        JSON string with cancellation result
    """
    itinerary = itineraries_db.get(itinerary_id)
    if not itinerary:
        return json.dumps({"error": f"Itinerary {itinerary_id} not found"})
    
    if itinerary.status == "cancelled":
        return json.dumps({"error": "Itinerary already cancelled"})
    
    itinerary.status = "cancelled"
    
    # Restore available seats
    for flight in itinerary.flights:
        flight.available_seats += 1
    
    return json.dumps({
        "success": True,
        "message": f"Itinerary {itinerary_id} cancelled successfully"
    })

def file_ticket(user_name: str, issue_description: str) -> str:
    """
    File a customer support ticket.
    
    Args:
        user_name: Name of the user filing the ticket
        issue_description: Description of the issue
    
    Returns:
        JSON string with ticket information
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    ticket_id = str(uuid.uuid4())
    
    ticket = Ticket(
        ticket_id=ticket_id,
        user_id=user.user_id,
        itinerary_id="",  # May not be related to specific itinerary
        confirmation_number="",
        issue_description=issue_description,
        status="pending"
    )
    
    tickets_db[ticket_id] = ticket
    
    return json.dumps({
        "success": True,
        "ticket_id": ticket_id,
        "message": f"Support ticket filed successfully. Ticket ID: {ticket_id}"
    })

## DSPy Configuration

Set up the language model and configure DSPy.

In [91]:
# Configure OpenAI API key (set this in your environment)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Initialize the language model
# lm = dspy.LM('openai/gpt-4o-mini')
lm = dspy.LM('groq/openai/gpt-oss-120b')
dspy.configure(lm=lm)

## Customer Service Agent Signature

Define the DSPy signature for our customer service agent.

In [92]:
class DSPyAirlineCustomerService(dspy.Signature):
    """
    You are an airline customer service agent that helps users book and manage flights.
    You are given a list of tools to handle user requests, and you should decide the right tool to use
    in order to fulfill users' requests.
    
    Available tools:
    - fetch_flight_info: Get available flights for a route and date
    - pick_flight: Select the best flight based on duration or price
    - get_user_info: Retrieve user profile information
    - book_flight: Book a flight for a user
    - cancel_itinerary: Cancel an existing booking
    - file_ticket: File a customer support ticket
    
    Always be helpful, professional, and provide clear information about flights and bookings.
    """
    
    user_request = dspy.InputField(desc="The user's request or question")
    response = dspy.OutputField(desc="Your response to the user")

## Create the ReAct Agent

Initialize the DSPy ReAct agent with our tools.

In [93]:
# Define the tools for the agent
tools = [
    fetch_flight_info,
    pick_flight,
    get_user_info,
    book_flight,
    cancel_itinerary,
    file_ticket
]

# Create the ReAct agent
agent = dspy.ReAct(
    signature=DSPyAirlineCustomerService,
    tools=tools,
    max_iters=10
)

---

# 🚀 Otimização Avançada do Agente

## 🎯 Por que Otimizar um Agente?

Agentes de atendimento ao cliente têm requisitos únicos que beneficiam muito de otimização:

1. **✅ Task Completion**: O agente deve completar tarefas corretamente
2. **💬 Qualidade de Resposta**: Respostas devem ser úteis e profissionais
3. **⚡ Eficiência**: Usar ferramentas de forma eficiente (menos iterações)
4. **🎯 Precisão**: Escolher as ferramentas corretas para cada situação
5. **🔄 Robustez**: Lidar com casos edge e erros graciosamente

### 🔬 Técnicas que Vamos Aplicar

- **MIPRO**: Otimização avançada de instruções e exemplos
- **Multi-métrica**: Balancear múltiplos objetivos simultaneamente
- **Ensemble**: Combinar múltiplos modelos para robustez
- **Dataset Few-Shot**: Criar exemplos de treinamento de alta qualidade

### 📊 Métricas para Agentes

Diferente de tarefas simples de classificação, agentes precisam de métricas mais sofisticadas:

1. **Task Success Rate**: Taxa de conclusão bem-sucedida de tarefas
2. **Response Quality**: Qualidade e utilidade das respostas
3. **Tool Efficiency**: Número de chamadas de ferramentas necessárias
4. **Error Handling**: Como lida com erros e casos edge
5. **User Satisfaction**: Simulação de satisfação do usuário


In [94]:
# 🎯 Criando Dataset de Treinamento e Validação para Otimização

import time
import numpy as np
from typing import Dict, Any

print("📚 Criando Dataset Few-Shot para Otimização do Agente")
print("="*60)

# Dataset de treinamento: exemplos que demonstram diferentes cenários
# Estes exemplos servem como few-shot demonstrations para o agente aprender

agent_training_examples = [
    # Cenário 1: Booking simples
    {
        "user_request": "I need to book a flight from SFO to JFK on 2025-09-01. My name is Adam.",
        "expected_tools_used": ["fetch_flight_info", "book_flight"],
        "expected_outcome": "booking_confirmed",
        "expected_response_contains": ["confirmation", "flight", "booked"],
        "task_type": "booking",
        "complexity": "simple"
    },
    
    # Cenário 2: Consulta de voos
    {
        "user_request": "What flights are available from JFK to LAX?",
        "expected_tools_used": ["fetch_flight_info"],
        "expected_outcome": "flights_listed",
        "expected_response_contains": ["flight", "available", "departure", "arrival"],
        "task_type": "query",
        "complexity": "simple"
    },
    
    # Cenário 3: Buscar informações do usuário
    {
        "user_request": "Can you tell me my profile information? My name is Sarah.",
        "expected_tools_used": ["get_user_info"],
        "expected_outcome": "user_info_retrieved",
        "expected_response_contains": ["name", "email", "phone"],
        "task_type": "profile_lookup",
        "complexity": "simple"
    },
    
    # Cenário 4: File ticket
    {
        "user_request": "I need to file a complaint about a delayed flight. My name is Adam.",
        "expected_tools_used": ["file_ticket"],
        "expected_outcome": "ticket_created",
        "expected_response_contains": ["ticket", "filed", "complaint"],
        "task_type": "support",
        "complexity": "simple"
    },
    
    # Cenário 5: Booking com preferência
    {
        "user_request": "I want the cheapest flight from SFO to JFK on 2025-09-01. I'm Adam.",
        "expected_tools_used": ["fetch_flight_info", "pick_flight", "book_flight"],
        "expected_outcome": "booking_confirmed",
        "expected_response_contains": ["cheapest", "price", "booked"],
        "task_type": "booking",
        "complexity": "medium"
    },
    
    # Cenário 6: Cancelamento
    {
        "user_request": "I need to cancel my booking. My itinerary ID is test_123.",
        "expected_tools_used": ["cancel_itinerary"],
        "expected_outcome": "cancellation_successful",
        "expected_response_contains": ["cancel", "cancelled"],
        "task_type": "cancellation",
        "complexity": "medium"
    },
    
    # Cenário 7: Query complexa
    {
        "user_request": "Show me all available flights from San Francisco to New York for next week.",
        "expected_tools_used": ["fetch_flight_info"],
        "expected_outcome": "flights_listed",
        "expected_response_contains": ["flight", "SFO", "JFK"],
        "task_type": "query",
        "complexity": "medium"
    },
    
    # Cenário 8: Múltiplas ações
    {
        "user_request": "Hi, I'm Sarah. Can you check my profile and then show me flights to LAX?",
        "expected_tools_used": ["get_user_info", "fetch_flight_info"],
        "expected_outcome": "multiple_actions_completed",
        "expected_response_contains": ["profile", "flight"],
        "task_type": "multi_step",
        "complexity": "complex"
    },
]

# Dataset de validação/teste
agent_test_examples = [
    {
        "user_request": "Book me the fastest flight from SFO to JFK on 2025-10-15. Name is Adam.",
        "expected_tools_used": ["fetch_flight_info", "pick_flight", "book_flight"],
        "expected_outcome": "booking_confirmed",
        "expected_response_contains": ["fastest", "duration", "booked"],
        "task_type": "booking",
        "complexity": "medium"
    },
    {
        "user_request": "What's my frequent flyer number? I'm Sarah.",
        "expected_tools_used": ["get_user_info"],
        "expected_outcome": "user_info_retrieved",
        "expected_response_contains": ["frequent", "flyer"],
        "task_type": "profile_lookup",
        "complexity": "simple"
    },
    {
        "user_request": "I want to file a support ticket for lost luggage. My name is Adam.",
        "expected_tools_used": ["file_ticket"],
        "expected_outcome": "ticket_created",
        "expected_response_contains": ["ticket", "luggage"],
        "task_type": "support",
        "complexity": "simple"
    },
]

# Converter para formato DSPy
agent_train_dataset = [
    dspy.Example(**ex).with_inputs('user_request') 
    for ex in agent_training_examples
]

agent_test_dataset = [
    dspy.Example(**ex).with_inputs('user_request') 
    for ex in agent_test_examples
]

print(f"✅ Dataset criado:")
print(f"  • Treinamento: {len(agent_train_dataset)} exemplos")
print(f"  • Teste: {len(agent_test_dataset)} exemplos")
print(f"\n📊 Distribuição por tipo de tarefa:")
from collections import Counter
task_types = Counter([ex['task_type'] for ex in agent_training_examples])
for task_type, count in task_types.items():
    print(f"  • {task_type}: {count}")


📚 Criando Dataset Few-Shot para Otimização do Agente
✅ Dataset criado:
  • Treinamento: 8 exemplos
  • Teste: 3 exemplos

📊 Distribuição por tipo de tarefa:
  • booking: 2
  • query: 2
  • profile_lookup: 1
  • support: 1
  • cancellation: 1
  • multi_step: 1


In [95]:
# 📊 Definindo Métricas Multi-objetivo para o Agente

print("📊 Definindo Métricas para Otimização do Agente")
print("="*60)

def extract_tools_used(trace):
    """Extrai ferramentas usadas do trace do agente."""
    tools_used = []
    if hasattr(trace, 'trace') and trace.trace:
        for step in trace.trace:
            if hasattr(step, 'tool') and step.tool:
                tools_used.append(step.tool)
    return tools_used

def task_completion_metric(example, pred, trace=None):
    """
    Métrica 1: Task Completion (40% peso)
    Verifica se a tarefa foi completada com sucesso.
    """
    response = pred.response.lower() if hasattr(pred, 'response') else ""
    
    # Verificar se a resposta contém elementos esperados
    contains_expected = all(
        keyword.lower() in response 
        for keyword in example.get('expected_response_contains', [])
    )
    
    # Verificar se não há erros explícitos
    error_keywords = ["error", "sorry", "cannot", "unable", "failed"]
    has_errors = any(keyword in response for keyword in error_keywords)
    
    # Score baseado em se contém esperado e não tem erros
    if contains_expected and not has_errors:
        return 1.0
    elif contains_expected:
        return 0.7  # Completou mas com warnings
    elif not has_errors:
        return 0.5  # Sem erros mas não completa
    else:
        return 0.2  # Erros evidentes

def tool_usage_accuracy_metric(example, pred, trace=None):
    """
    Métrica 2: Tool Usage Accuracy (25% peso)
    Verifica se as ferramentas corretas foram usadas.
    """
    tools_used = extract_tools_used(trace) if trace else []
    expected_tools = example.get('expected_tools_used', [])
    
    if not expected_tools:
        return 1.0  # Não há expectativa específica
    
    # Verificar se todas as ferramentas esperadas foram usadas
    tools_used_set = set(tools_used)
    expected_set = set(expected_tools)
    
    # Score baseado em interseção
    if expected_set.issubset(tools_used_set):
        return 1.0  # Todas as ferramentas esperadas foram usadas
    elif len(expected_set & tools_used_set) > 0:
        # Algumas ferramentas corretas foram usadas
        overlap = len(expected_set & tools_used_set)
        return overlap / len(expected_set)
    else:
        return 0.0  # Nenhuma ferramenta esperada foi usada

def response_quality_metric(example, pred, trace=None):
    """
    Métrica 3: Response Quality (20% peso)
    Avalia qualidade, profissionalismo e utilidade da resposta.
    """
    response = pred.response if hasattr(pred, 'response') else ""
    
    if not response:
        return 0.0
    
    score = 0.0
    
    # Comprimento apropriado (não muito curto, não muito longo)
    word_count = len(response.split())
    if 20 <= word_count <= 200:
        score += 0.3
    elif 10 <= word_count < 20 or 200 < word_count <= 300:
        score += 0.2
    else:
        score += 0.1
    
    # Profissionalismo (contém saudações, é cortês)
    professional_keywords = ["thank", "please", "help", "assist", "welcome"]
    has_professional = any(kw in response.lower() for kw in professional_keywords)
    if has_professional:
        score += 0.3
    
    # Informação útil (contém detalhes específicos)
    detail_keywords = ["flight", "confirmation", "price", "time", "date", "number"]
    has_details = sum(1 for kw in detail_keywords if kw in response.lower())
    score += min(has_details * 0.1, 0.4)  # Máximo 0.4
    
    return min(score, 1.0)

def efficiency_metric(example, pred, trace=None):
    """
    Métrica 4: Efficiency (15% peso)
    Penaliza uso excessivo de ferramentas ou iterações.
    """
    tools_used = extract_tools_used(trace) if trace else []
    expected_tools = example.get('expected_tools_used', [])
    
    if not tools_used:
        return 0.5  # Sem ferramentas, pode não ter completado
    
    # Penalizar uso excessivo de ferramentas
    optimal_count = len(expected_tools) if expected_tools else 2
    tool_count = len(tools_used)
    
    if tool_count <= optimal_count:
        return 1.0
    elif tool_count <= optimal_count + 1:
        return 0.8
    elif tool_count <= optimal_count + 2:
        return 0.6
    else:
        return 0.4  # Muitas ferramentas desnecessárias

def agent_multi_objective_metric(example, pred, trace=None, weights=None):
    """
    Métrica combinada multi-objetivo para agentes.
    
    Pesos padrão:
    - Task Completion: 40%
    - Tool Accuracy: 25%
    - Response Quality: 20%
    - Efficiency: 15%
    """
    if weights is None:
        weights = {
            'task_completion': 0.4,
            'tool_accuracy': 0.25,
            'response_quality': 0.2,
            'efficiency': 0.15
        }
    
    # Calcular cada métrica
    task_score = task_completion_metric(example, pred, trace)
    tool_score = tool_usage_accuracy_metric(example, pred, trace)
    quality_score = response_quality_metric(example, pred, trace)
    efficiency_score = efficiency_metric(example, pred, trace)
    
    # Combinar com pesos
    combined_score = (
        task_score * weights['task_completion'] +
        tool_score * weights['tool_accuracy'] +
        quality_score * weights['response_quality'] +
        efficiency_score * weights['efficiency']
    )
    
    return combined_score

print("✅ Métricas definidas:")
print("  1. Task Completion (40%) - Tarefa completada com sucesso")
print("  2. Tool Usage Accuracy (25%) - Ferramentas corretas usadas")
print("  3. Response Quality (20%) - Qualidade e profissionalismo")
print("  4. Efficiency (15%) - Uso eficiente de ferramentas")


📊 Definindo Métricas para Otimização do Agente
✅ Métricas definidas:
  1. Task Completion (40%) - Tarefa completada com sucesso
  2. Tool Usage Accuracy (25%) - Ferramentas corretas usadas
  3. Response Quality (20%) - Qualidade e profissionalismo
  4. Efficiency (15%) - Uso eficiente de ferramentas


In [96]:
# 🔍 Avaliação Baseline do Agente

print("🔍 Avaliando Agente Baseline (Sem Otimização)")
print("="*60)

def evaluate_agent(agent_model, test_examples, model_name="Agent"):
    """Avalia agente em métricas múltiplas."""
    results = {
        'task_completion': [],
        'tool_accuracy': [],
        'response_quality': [],
        'efficiency': [],
        'combined': [],
        'detailed_results': []
    }
    
    print(f"\n📊 Testando {model_name} em {len(test_examples)} exemplos...\n")
    
    for i, example in enumerate(test_examples, 1):
        try:
            # Executar agente
            result = agent_model(user_request=example.user_request)
            
            # Calcular métricas individuais
            task_score = task_completion_metric(example, result, trace=None)
            tool_score = tool_usage_accuracy_metric(example, result, trace=None)
            quality_score = response_quality_metric(example, result, trace=None)
            efficiency_score = efficiency_metric(example, result, trace=None)
            combined_score = agent_multi_objective_metric(example, result, trace=None)
            
            # Armazenar resultados
            results['task_completion'].append(task_score)
            results['tool_accuracy'].append(tool_score)
            results['response_quality'].append(quality_score)
            results['efficiency'].append(efficiency_score)
            results['combined'].append(combined_score)
            
            results['detailed_results'].append({
                'request': example.user_request[:60] + "...",
                'task': task_score,
                'tool': tool_score,
                'quality': quality_score,
                'efficiency': efficiency_score,
                'combined': combined_score
            })
            
            print(f"  {i}. '{example.user_request[:50]}...'")
            print(f"     📊 Score: {combined_score:.3f} (Task: {task_score:.2f}, Tool: {tool_score:.2f}, Quality: {quality_score:.2f}, Eff: {efficiency_score:.2f})")
            
        except Exception as e:
            print(f"  ❌ Erro no exemplo {i}: {e}")
            # Adicionar scores zero em caso de erro
            results['task_completion'].append(0.0)
            results['tool_accuracy'].append(0.0)
            results['response_quality'].append(0.0)
            results['efficiency'].append(0.0)
            results['combined'].append(0.0)
    
    # Calcular médias
    return {
        'task_completion': np.mean(results['task_completion']),
        'tool_accuracy': np.mean(results['tool_accuracy']),
        'response_quality': np.mean(results['response_quality']),
        'efficiency': np.mean(results['efficiency']),
        'combined': np.mean(results['combined']),
        'detailed': results['detailed_results']
    }

# Avaliar agente baseline
baseline_results = evaluate_agent(agent, agent_test_dataset, "Baseline Agent")

print(f"\n📈 Resultados Baseline:")
print(f"  🎯 Task Completion:  {baseline_results['task_completion']:.3f}")
print(f"  🔧 Tool Accuracy:    {baseline_results['tool_accuracy']:.3f}")
print(f"  💬 Response Quality: {baseline_results['response_quality']:.3f}")
print(f"  ⚡ Efficiency:        {baseline_results['efficiency']:.3f}")
print(f"  📊 Score Combinado:  {baseline_results['combined']:.3f}")


🔍 Avaliando Agente Baseline (Sem Otimização)

📊 Testando Baseline Agent em 3 exemplos...

  1. 'Book me the fastest flight from SFO to JFK on 2025...'
     📊 Score: 0.475 (Task: 0.50, Tool: 0.00, Quality: 1.00, Eff: 0.50)
  2. 'What's my frequent flyer number? I'm Sarah....'
     📊 Score: 0.575 (Task: 1.00, Tool: 0.00, Quality: 0.50, Eff: 0.50)
  3. 'I want to file a support ticket for lost luggage. ...'
     📊 Score: 0.595 (Task: 1.00, Tool: 0.00, Quality: 0.60, Eff: 0.50)

📈 Resultados Baseline:
  🎯 Task Completion:  0.833
  🔧 Tool Accuracy:    0.000
  💬 Response Quality: 0.700
  ⚡ Efficiency:        0.500
  📊 Score Combinado:  0.548


In [97]:
# 🚀 Otimização com BootstrapFewShot

print("\n🚀 PASSO 1: Otimização com BootstrapFewShot")
print("="*60)

# Criar otimizador BootstrapFewShot
bootstrap_optimizer = dspy.BootstrapFewShot(
    metric=lambda ex, pred, trace=None: agent_multi_objective_metric(ex, pred, trace),
    max_bootstrapped_demos=6,
    max_labeled_demos=12
)

print("🔧 Compilando agente otimizado com BootstrapFewShot...")
print("💡 Isso pode levar alguns minutos enquanto o otimizador:")
print("   • Gera exemplos few-shot automaticamente")
print("   • Seleciona os melhores exemplos")
print("   • Otimiza para as métricas definidas")

start_time = time.time()

try:
    optimized_agent_bootstrap = bootstrap_optimizer.compile(
        dspy.ReAct(
            signature=DSPyAirlineCustomerService,
            tools=tools,
            max_iters=10
        ),
        trainset=agent_train_dataset
    )
    
    bootstrap_time = time.time() - start_time
    print(f"✅ Otimização concluída em {bootstrap_time:.2f}s")
    
    # Avaliar agente otimizado
    bootstrap_results = evaluate_agent(
        optimized_agent_bootstrap, 
        agent_test_dataset, 
        "BootstrapFewShot Optimized"
    )
    
    print(f"\n📊 Comparação: Baseline vs BootstrapFewShot")
    print(f"  🎯 Task Completion:  {baseline_results['task_completion']:.3f} → {bootstrap_results['task_completion']:.3f} ({bootstrap_results['task_completion'] - baseline_results['task_completion']:+.3f})")
    print(f"  🔧 Tool Accuracy:    {baseline_results['tool_accuracy']:.3f} → {bootstrap_results['tool_accuracy']:.3f} ({bootstrap_results['tool_accuracy'] - baseline_results['tool_accuracy']:+.3f})")
    print(f"  💬 Response Quality: {baseline_results['response_quality']:.3f} → {bootstrap_results['response_quality']:.3f} ({bootstrap_results['response_quality'] - baseline_results['response_quality']:+.3f})")
    print(f"  ⚡ Efficiency:        {baseline_results['efficiency']:.3f} → {bootstrap_results['efficiency']:.3f} ({bootstrap_results['efficiency'] - baseline_results['efficiency']:+.3f})")
    print(f"  📊 Score Combinado:  {baseline_results['combined']:.3f} → {bootstrap_results['combined']:.3f} ({bootstrap_results['combined'] - baseline_results['combined']:+.3f})")
    
except Exception as e:
    print(f"⚠️ Erro na otimização BootstrapFewShot: {e}")
    print("   Continuando com outras técnicas...")
    bootstrap_results = None



🚀 PASSO 1: Otimização com BootstrapFewShot
🔧 Compilando agente otimizado com BootstrapFewShot...
💡 Isso pode levar alguns minutos enquanto o otimizador:
   • Gera exemplos few-shot automaticamente
   • Seleciona os melhores exemplos
   • Otimiza para as métricas definidas


 75%|███████▌  | 6/8 [00:12<00:04,  2.10s/it]


Bootstrapped 6 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.
✅ Otimização concluída em 12.61s

📊 Testando BootstrapFewShot Optimized em 3 exemplos...



Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python

  1. 'Book me the fastest flight from SFO to JFK on 2025...'
     📊 Score: 0.475 (Task: 0.50, Tool: 0.00, Quality: 1.00, Eff: 0.50)
  2. 'What's my frequent flyer number? I'm Sarah....'
     📊 Score: 0.635 (Task: 1.00, Tool: 0.00, Quality: 0.80, Eff: 0.50)
  3. 'I want to file a support ticket for lost luggage. ...'
     📊 Score: 0.635 (Task: 1.00, Tool: 0.00, Quality: 0.80, Eff: 0.50)

📊 Comparação: Baseline vs BootstrapFewShot
  🎯 Task Completion:  0.833 → 0.833 (+0.000)
  🔧 Tool Accuracy:    0.000 → 0.000 (+0.000)
  💬 Response Quality: 0.700 → 0.867 (+0.167)
  ⚡ Efficiency:        0.500 → 0.500 (+0.000)
  📊 Score Combinado:  0.548 → 0.582 (+0.033)


In [98]:
# 🔬 Otimização Avançada com MIPRO

print("\n🔬 PASSO 2: Otimização Avançada com MIPRO")
print("="*60)
print("💡 MIPRO vai:")
print("   • Otimizar instruções do agente")
print("   • Selecionar melhores exemplos few-shot")
print("   • Refinar baseado em erros")
print("   • Encontrar configuração ótima")

try:
    mipro_optimizer = dspy.MIPRO(
        metric=lambda ex, pred, trace=None: agent_multi_objective_metric(ex, pred, trace),
        num_candidates=8,  # Número de instruções candidatas
        init_temperature=1.0
    )
    
    print("🔧 Compilando com MIPRO (isso pode levar vários minutos)...")
    start_time = time.time()
    
    optimized_agent_mipro = mipro_optimizer.compile(
        dspy.ReAct(
            signature=DSPyAirlineCustomerService,
            tools=tools,
            max_iters=10
        ),
        trainset=agent_train_dataset
    )
    
    mipro_time = time.time() - start_time
    print(f"✅ Otimização MIPRO concluída em {mipro_time:.2f}s")
    
    # Avaliar agente otimizado com MIPRO
    mipro_results = evaluate_agent(
        optimized_agent_mipro,
        agent_test_dataset,
        "MIPRO Optimized"
    )
    
    print(f"\n📊 Comparação: Baseline vs MIPRO")
    print(f"  🎯 Task Completion:  {baseline_results['task_completion']:.3f} → {mipro_results['task_completion']:.3f} ({mipro_results['task_completion'] - baseline_results['task_completion']:+.3f})")
    print(f"  🔧 Tool Accuracy:    {baseline_results['tool_accuracy']:.3f} → {mipro_results['tool_accuracy']:.3f} ({mipro_results['tool_accuracy'] - baseline_results['tool_accuracy']:+.3f})")
    print(f"  💬 Response Quality: {baseline_results['response_quality']:.3f} → {mipro_results['response_quality']:.3f} ({mipro_results['response_quality'] - baseline_results['response_quality']:+.3f})")
    print(f"  ⚡ Efficiency:        {baseline_results['efficiency']:.3f} → {mipro_results['efficiency']:.3f} ({mipro_results['efficiency'] - baseline_results['efficiency']:+.3f})")
    print(f"  📊 Score Combinado:  {baseline_results['combined']:.3f} → {mipro_results['combined']:.3f} ({mipro_results['combined'] - baseline_results['combined']:+.3f})")
    
    if bootstrap_results:
        print(f"\n📊 Comparação: BootstrapFewShot vs MIPRO")
        print(f"  📊 Score Combinado:  {bootstrap_results['combined']:.3f} → {mipro_results['combined']:.3f} ({mipro_results['combined'] - bootstrap_results['combined']:+.3f})")
    
except Exception as e:
    print(f"⚠️ MIPRO pode não estar disponível nesta versão do DSPy")
    print(f"   Erro: {e}")
    print(f"\n💡 Alternativa: Usando BootstrapFewShotWithRandomSearch")
    
    try:
        random_search_optimizer = dspy.BootstrapFewShotWithRandomSearch(
            metric=lambda ex, pred, trace=None: agent_multi_objective_metric(ex, pred, trace),
            max_bootstrapped_demos=6,
            max_labeled_demos=12,
            num_candidate_programs=8
        )
        
        print("🔧 Compilando com RandomSearch...")
        start_time = time.time()
        
        optimized_agent_rs = random_search_optimizer.compile(
            dspy.ReAct(
                signature=DSPyAirlineCustomerService,
                tools=tools,
                max_iters=10
            ),
            trainset=agent_train_dataset
        )
        
        rs_time = time.time() - start_time
        print(f"✅ Otimização RandomSearch concluída em {rs_time:.2f}s")
        
        rs_results = evaluate_agent(
            optimized_agent_rs,
            agent_test_dataset,
            "RandomSearch Optimized"
        )
        
        print(f"\n📊 Comparação Final:")
        print(f"  Baseline:    {baseline_results['combined']:.3f}")
        if bootstrap_results:
            print(f"  Bootstrap:   {bootstrap_results['combined']:.3f}")
        print(f"  RandomSearch: {rs_results['combined']:.3f}")
        
    except Exception as e2:
        print(f"⚠️ Erro também no RandomSearch: {e2}")



🔬 PASSO 2: Otimização Avançada com MIPRO
💡 MIPRO vai:
   • Otimizar instruções do agente
   • Selecionar melhores exemplos few-shot
   • Refinar baseado em erros
   • Encontrar configuração ótima
⚠️ MIPRO pode não estar disponível nesta versão do DSPy
   Erro: module 'dspy' has no attribute 'MIPRO'

💡 Alternativa: Usando BootstrapFewShotWithRandomSearch
Going to sample between 1 and 6 traces per predictor.
Will attempt to bootstrap 8 candidate sets.
🔧 Compilando com RandomSearch...
Average Metric: 2.26 / 4 (56.5%):  50%|█████     | 4/8 [00:01<00:01,  2.06it/s]

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python

Average Metric: 4.72 / 8 (59.0%): 100%|██████████| 8/8 [00:04<00:00,  1.79it/s]

2025/11/01 17:09:26 INFO dspy.evaluate.evaluate: Average Metric: 4.720000000000001 / 8 (59.0%)



New best score: 59.0 for seed -3
Scores so far: [59.0]
Best score so far: 59.0
Average Metric: 4.62 / 8 (57.8%): 100%|██████████| 8/8 [00:03<00:00,  2.43it/s]

...

Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Average Metric: 3.11 / 6 (51.8%):  75%|███████▌  | 6/8 [00:03<00:00,  3.04it/s]

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python

Average Metric: 3.94 / 8 (49.2%): 100%|██████████| 8/8 [00:03<00:00,  2.06it/s]

2025/11/01 17:11:07 INFO dspy.evaluate.evaluate: Average Metric: 3.94 / 8 (49.2%)



Scores so far: [59.0, 57.75, 60.75, 56.5, 62.5, 55.0, 57.75, 53.5, 61.5, 47.5, 49.25]
Best score so far: 62.5
11 candidate programs found.
✅ Otimização RandomSearch concluída em 105.92s

📊 Testando RandomSearch Optimized em 3 exemplos...

  1. 'Book me the fastest flight from SFO to JFK on 2025...'
     📊 Score: 0.315 (Task: 0.20, Tool: 0.00, Quality: 0.80, Eff: 0.50)
  2. 'What's my frequent flyer number? I'm Sarah....'
     📊 Score: 0.575 (Task: 1.00, Tool: 0.00, Quality: 0.50, Eff: 0.50)
  3. 'I want to file a support ticket for lost luggage. ...'
     📊 Score: 0.515 (Task: 0.70, Tool: 0.00, Quality: 0.80, Eff: 0.50)

📊 Comparação Final:
  Baseline:    0.548
  Bootstrap:   0.582
  RandomSearch: 0.468


## 💾 Serialização e Persistência do Modelo Otimizado

### 🎯 Por que Serializar?

Após otimizar o agente (que pode levar minutos ou horas), você **não quer** re-otimizar toda vez que inicializar o sistema. A serialização permite:

1. **⏱️ Economia de Tempo**: Evita re-otimização desnecessária
2. **💰 Economia de Recursos**: Reduz chamadas ao LLM e custos
3. **🔄 Consistência**: Usa sempre o mesmo modelo otimizado
4. **🚀 Deploy Rápido**: Carrega modelo pronto para produção

### 📦 O que é Salvo?

Quando você serializa um modelo DSPy otimizado, são salvos:

- ✅ **Exemplos Few-Shot Selecionados**: Os melhores exemplos escolhidos pelo otimizador
- ✅ **Instruções Otimizadas**: Instruções refinadas pelo MIPRO (se usado)
- ✅ **Configuração do Modelo**: Signature, tools, parâmetros
- ✅ **Métricas de Performance**: Informações sobre o desempenho

### 🗂️ Formato de Serialização

O DSPy usa formato JSON para serializar modelos. Isso permite:
- Fácil inspeção e debug
- Versionamento
- Compatibilidade entre ambientes
- Armazenamento em qualquer sistema de arquivos


In [ ]:
# 💾 Serializando o Modelo Otimizado (Usando Métodos Oficiais do DSPy)

print("💾 Serializando Modelo Otimizado com DSPy.save()")
print("="*60)

from pathlib import Path
import json

# Criar diretório para modelos salvos
models_dir = Path("saved_models")
models_dir.mkdir(exist_ok=True)

# Determinar qual modelo otimizado salvar
model_to_save = None
model_name = None

if 'optimized_agent_mipro' in locals():
    model_to_save = optimized_agent_mipro
    model_name = "agent_mipro_optimized"
    print(f"✅ Salvando modelo otimizado com MIPRO")
elif 'optimized_agent_bootstrap' in locals():
    model_to_save = optimized_agent_bootstrap
    model_name = "agent_bootstrap_optimized"
    print(f"✅ Salvando modelo otimizado com BootstrapFewShot")
else:
    print("⚠️ Nenhum modelo otimizado encontrado. Pulando serialização.")
    print("   Execute as células de otimização primeiro!")

if model_to_save:
    try:
        # MÉTODO 1: Salvar apenas o estado (estado + configuração, sem estrutura)
        # Útil quando você quer manter o código separado
        state_path = models_dir / f"{model_name}_state.json"
        print(f"\n📦 Método 1: Salvando apenas estado...")
        model_to_save.save(str(state_path), save_program=False)
        print(f"   ✅ Estado salvo em: {state_path}")
        print(f"   📊 Tamanho: {state_path.stat().st_size / 1024:.2f} KB")
        print(f"   💡 Para carregar: Reconstruir estrutura + agent.load('{state_path}')")
        
        # MÉTODO 2: Salvar programa completo (estrutura + estado + configuração)
        # Mais conveniente - não precisa reconstruir manualmente
        full_path = models_dir / f"{model_name}_full"
        full_path.mkdir(exist_ok=True)
        print(f"\n📦 Método 2: Salvando programa completo...")
        model_to_save.save(str(full_path), save_program=True)   
        
        # Verificar o que foi salvo
        saved_files = list(full_path.rglob("*"))
        print(f"   ✅ Programa completo salvo em: {full_path}/")
        print(f"   📁 Arquivos salvos: {len(saved_files)}")
        total_size = sum(f.stat().st_size for f in saved_files if f.is_file())
        print(f"   📊 Tamanho total: {total_size / 1024:.2f} KB")
        print(f"   💡 Para carregar: dspy.load('{full_path}')")
        
        # Salvar também metadados adicionais (opcional, para rastreabilidade)
        metadata_path = models_dir / f"{model_name}_metadata.json"
        metadata = {
            'optimizer_type': 'MIPRO' if 'mipro' in model_name else 'BootstrapFewShot',
            'saved_at': datetime.now().isoformat(),
            'state_file': str(state_path),
            'full_program_dir': str(full_path),
            'model_signature': 'DSPyAirlineCustomerService',
            'tools': [tool.__name__ for tool in tools],
            'max_iters': 10,
        }
        
        # Adicionar informações de performance se disponíveis
        if 'baseline_results' in locals():
            metadata['performance'] = {
                'baseline': {
                    'task_completion': float(baseline_results.get('task_completion', 0)),
                    'tool_accuracy': float(baseline_results.get('tool_accuracy', 0)),
                    'response_quality': float(baseline_results.get('response_quality', 0)),
                    'efficiency': float(baseline_results.get('efficiency', 0)),
                    'combined': float(baseline_results.get('combined', 0))
                }
            }
            
            optimized_results = mipro_results if 'optimized_agent_mipro' in locals() else bootstrap_results
            if optimized_results:
                metadata['performance']['optimized'] = {
                    'task_completion': float(optimized_results.get('task_completion', 0)),
                    'tool_accuracy': float(optimized_results.get('tool_accuracy', 0)),
                    'response_quality': float(optimized_results.get('response_quality', 0)),
                    'efficiency': float(optimized_results.get('efficiency', 0)),
                    'combined': float(optimized_results.get('combined', 0))
                }
                metadata['performance']['improvement'] = {
                    'task_completion': float(optimized_results.get('task_completion', 0) - baseline_results.get('task_completion', 0)),
                    'tool_accuracy': float(optimized_results.get('tool_accuracy', 0) - baseline_results.get('tool_accuracy', 0)),
                    'response_quality': float(optimized_results.get('response_quality', 0) - baseline_results.get('response_quality', 0)),
                    'efficiency': float(optimized_results.get('efficiency', 0) - baseline_results.get('efficiency', 0)),
                    'combined': float(optimized_results.get('combined', 0) - baseline_results.get('combined', 0))
                }
        
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        print(f"\n📊 Metadados salvos em: {metadata_path}")
        print(f"\n✅ Serialização completa! Use um dos métodos acima para carregar.")
        
    except Exception as e:
        print(f"⚠️ Erro ao serializar modelo: {e}")
        import traceback
        traceback.print_exc()
        print(f"\n💡 Verifique se o modelo é um objeto DSPy válido com método .save()")
else:
    print("\n💡 Para serializar:")
    print("   1. Execute as células de otimização primeiro")
    print("   2. Depois execute esta célula novamente")


💾 Serializando Modelo Otimizado com DSPy.save()
✅ Salvando modelo otimizado com BootstrapFewShot

📦 Método 1: Salvando apenas estado...
   ✅ Estado salvo em: saved_models/agent_bootstrap_optimized_state.json
   📊 Tamanho: 28.23 KB
   💡 Para carregar: Reconstruir estrutura + agent.load('saved_models/agent_bootstrap_optimized_state.json')

📦 Método 2: Salvando programa completo...
   ✅ Programa completo salvo em: saved_models/agent_bootstrap_optimized_full/
   📁 Arquivos salvos: 2
   📊 Tamanho total: 62.22 KB
   💡 Para carregar: dspy.load('saved_models/agent_bootstrap_optimized_full')

📊 Metadados salvos em: saved_models/agent_bootstrap_optimized_metadata.json

✅ Serialização completa! Use um dos métodos acima para carregar.


In [100]:
# 📂 Carregando Modelo Otimizado (Usando Métodos Oficiais do DSPy)

print("📂 Carregando Modelo Otimizado com DSPy.load()")
print("="*60)

def load_optimized_agent(model_name="agent_mipro_optimized", method="full"):
    """
    Carrega um agente otimizado salvo anteriormente.
    
    Args:
        model_name: Nome base do modelo (sem sufixo _state ou _full)
        method: "full" para carregar programa completo, "state" para apenas estado
    
    Returns:
        Agente otimizado carregado
    """
    if method == "full":
        # MÉTODO 1: Carregar programa completo (mais fácil - não precisa reconstruir)
        model_path = models_dir / f"{model_name}_full"
        
        if not model_path.exists():
            print(f"❌ Diretório do modelo não encontrado: {model_path}")
            print(f"   Verifique se o modelo foi salvo com save_program=True")
            return None
        
        try:
            print(f"🔄 Carregando programa completo de: {model_path}")
            loaded_agent = dspy.load(str(model_path))
            print(f"✅ Modelo carregado com sucesso!")
            return loaded_agent
        
        except Exception as e:
            print(f"❌ Erro ao carregar programa completo: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    elif method == "state":
        # MÉTODO 2: Carregar apenas estado (precisa reconstruir estrutura primeiro)
        state_path = models_dir / f"{model_name}_state.json"
        
        if not state_path.exists():
            print(f"❌ Arquivo de estado não encontrado: {state_path}")
            print(f"   Verifique se o modelo foi salvo com save_program=False")
            return None
        
        try:
            print(f"🔄 Carregando estado de: {state_path}")
            print(f"⚠️  Nota: Este método requer reconstruir a estrutura do agente")
            
            # Reconstruir estrutura do agente (mesma estrutura usada na otimização)
            agent_structure = dspy.ReAct(
                signature=DSPyAirlineCustomerService,
                tools=tools,
                max_iters=10
            )
            
            # Carregar estado otimizado
            agent_structure.load(str(state_path))
            
            print(f"✅ Estado carregado e aplicado ao agente!")
            return agent_structure
        
        except Exception as e:
            print(f"❌ Erro ao carregar estado: {e}")
            import traceback
            traceback.print_exc()
            return None
    else:
        print(f"❌ Método inválido: {method}. Use 'full' ou 'state'")
        return None

# Exemplo de uso (descomente para testar)
print("\n💡 Como usar em produção:")
print("   " + "="*60)
print("\n📝 Método 1: Programa Completo (Recomendado)")
print("   " + "-"*60)
print("   import dspy")
print("   ")
print("   # Carregar modelo completo")
print("   agent = dspy.load('saved_models/agent_mipro_optimized_full')")
print("   ")
print("   # Usar diretamente")
print("   result = agent(user_request='Book me a flight...')")
print("   print(result.response)")

print("\n📝 Método 2: Apenas Estado (Requer reconstruir estrutura)")
print("   " + "-"*60)
print("   import dspy")
print("   ")
print("   # Reconstruir estrutura")
print("   agent = dspy.ReAct(DSPyAirlineCustomerService, tools=[...], max_iters=10)")
print("   ")
print("   # Carregar estado otimizado")
print("   agent.load('saved_models/agent_mipro_optimized_state.json')")
print("   ")
print("   # Usar normalmente")
print("   result = agent(user_request='Book me a flight...')")
print("   print(result.response)")

print("\n✅ Diferenças:")
print("   • Método 1 (full): Mais fácil, carrega tudo de uma vez")
print("   • Método 2 (state): Mais flexível, mas requer código para estrutura")


📂 Carregando Modelo Otimizado com DSPy.load()

💡 Como usar em produção:

📝 Método 1: Programa Completo (Recomendado)
   ------------------------------------------------------------
   import dspy
   
   # Carregar modelo completo
   agent = dspy.load('saved_models/agent_mipro_optimized_full')
   
   # Usar diretamente
   result = agent(user_request='Book me a flight...')
   print(result.response)

📝 Método 2: Apenas Estado (Requer reconstruir estrutura)
   ------------------------------------------------------------
   import dspy
   
   # Reconstruir estrutura
   agent = dspy.ReAct(DSPyAirlineCustomerService, tools=[...], max_iters=10)
   
   # Carregar estado otimizado
   agent.load('saved_models/agent_mipro_optimized_state.json')
   
   # Usar normalmente
   result = agent(user_request='Book me a flight...')
   print(result.response)

✅ Diferenças:
   • Método 1 (full): Mais fácil, carrega tudo de uma vez
   • Método 2 (state): Mais flexível, mas requer código para estrutura


In [101]:
# 🔄 Verificando Modelos Salvos

print("📋 Modelos Disponíveis para Carregamento")
print("="*60)

if models_dir.exists():
    # Verificar programas completos (diretórios)
    saved_full_programs = [d for d in models_dir.iterdir() if d.is_dir() and d.name.endswith("_full")]
    # Verificar estados (arquivos JSON)
    saved_states = list(models_dir.glob("*_state.json"))
    # Verificar metadados
    saved_metadata = list(models_dir.glob("*_metadata.json"))
    
    if saved_full_programs:
        print(f"\n✅ Programas completos salvos ({len(saved_full_programs)}):")
        for program_dir in saved_full_programs:
            model_name = program_dir.stem.replace("_full", "")
            files_count = len(list(program_dir.rglob("*")))
            total_size = sum(f.stat().st_size for f in program_dir.rglob("*") if f.is_file())
            print(f"   • {model_name}:")
            print(f"     └─ Diretório: {program_dir.name}/")
            print(f"     └─ Arquivos: {files_count}")
            print(f"     └─ Tamanho: {total_size / 1024:.2f} KB")
            print(f"     └─ Carregar: dspy.load('{program_dir}')")
    
    if saved_states:
        print(f"\n✅ Estados salvos ({len(saved_states)}):")
        for state_file in saved_states:
            model_name = state_file.stem.replace("_state", "")
            size_kb = state_file.stat().st_size / 1024
            print(f"   • {model_name}:")
            print(f"     └─ Arquivo: {state_file.name}")
            print(f"     └─ Tamanho: {size_kb:.2f} KB")
            print(f"     └─ Carregar: Reconstruir estrutura + .load('{state_file}')")
    
    if saved_metadata:
        print(f"\n📊 Metadados disponíveis ({len(saved_metadata)}):")
        for meta_file in saved_metadata:
            try:
                with open(meta_file, 'r', encoding='utf-8') as f:
                    info = json.load(f)
                model_name = meta_file.stem.replace("_metadata", "")
                print(f"   • {model_name}:")
                print(f"     └─ Otimizador: {info.get('optimizer_type', 'Unknown')}")
                print(f"     └─ Salvo em: {info.get('saved_at', 'Unknown')}")
                if 'performance' in info:
                    perf = info['performance']
                    if 'improvement' in perf:
                        imp = perf['improvement']
                        print(f"     └─ Melhoria: +{imp.get('combined', 0):.3f} (score combinado)")
            except:
                print(f"   • {meta_file.name}")
    
    if not saved_full_programs and not saved_states:
        print("\n⚠️ Nenhum modelo salvo encontrado.")
        print("   Execute a célula de serialização após otimização.")
else:
    print("⚠️ Diretório de modelos não existe ainda.")
    print("   Execute a otimização e serialização primeiro.")


📋 Modelos Disponíveis para Carregamento

✅ Programas completos salvos (1):
   • agent_bootstrap_optimized:
     └─ Diretório: agent_bootstrap_optimized_full/
     └─ Arquivos: 2
     └─ Tamanho: 62.22 KB
     └─ Carregar: dspy.load('saved_models/agent_bootstrap_optimized_full')

✅ Estados salvos (1):
   • agent_bootstrap_optimized:
     └─ Arquivo: agent_bootstrap_optimized_state.json
     └─ Tamanho: 28.23 KB
     └─ Carregar: Reconstruir estrutura + .load('saved_models/agent_bootstrap_optimized_state.json')

📊 Metadados disponíveis (1):
   • agent_bootstrap_optimized:
     └─ Otimizador: BootstrapFewShot
     └─ Salvo em: 2025-11-01T17:11:16.508491
     └─ Melhoria: +0.033 (score combinado)


## 🔄 Fluxo Completo: Otimização → Serialização → Carregamento

### 📋 Workflow Recomendado

```
┌─────────────────────────────────────────────────────────┐
│  1. DESENVOLVIMENTO / OTIMIZAÇÃO                       │
│  ─────────────────────────────────────────────────────  │
│  • Criar dataset de treinamento                        │
│  • Definir métricas                                     │
│  • Executar otimização (MIPRO/BootstrapFewShot)        │
│  • Avaliar resultados                                  │
│  • Iterar e refinar                                    │
└─────────────────────────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────┐
│  2. SERIALIZAÇÃO                                         │
│  ─────────────────────────────────────────────────────  │
│  • Salvar modelo otimizado (pickle)                     │
│  • Salvar metadados (JSON)                              │
│  • Salvar métricas de performance                       │
│  • Versionar modelo                                     │
└─────────────────────────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────┐
│  3. DEPLOY / PRODUÇÃO                                    │
│  ─────────────────────────────────────────────────────  │
│  • Copiar arquivo .pkl para ambiente de produção        │
│  • Carregar modelo uma vez na inicialização             │
│  • Usar modelo carregado para todas as predições        │
│  • Monitorar performance                                │
└─────────────────────────────────────────────────────────┘
```

### 💡 Melhores Práticas

1. **📁 Organização**
   ```
   saved_models/
   ├── agent_mipro_optimized_v1.pkl
   ├── agent_mipro_optimized_v1.json
   ├── agent_mipro_optimized_v1_performance.json
   ├── agent_mipro_optimized_v2.pkl  # Versão melhorada
   └── ...
   ```

2. **🔖 Versionamento**
   - Use timestamps ou números de versão nos nomes
   - Mantenha histórico de versões
   - Documente mudanças entre versões

3. **✅ Validação ao Carregar**
   - Sempre teste o modelo carregado
   - Compare performance com esperado
   - Verifique integridade dos dados

4. **🔄 Re-otimização**
   - Re-otimize apenas quando necessário:
     * Novo dataset disponível
     * Métricas degradando em produção
     * Novo requisito de performance
     * Mudanças no modelo base (LLM)


In [102]:
# 🧪 Exemplo: Testando Modelo Carregado

print("🧪 Testando Modelo Carregado (se disponível)")
print("="*60)

# Tentar carregar modelo salvo usando método completo (recomendado)
loaded_test_agent = None
if models_dir.exists():
    saved_full_programs = [d for d in models_dir.iterdir() if d.is_dir() and d.name.endswith("_full")]
    
    if saved_full_programs:
        # Tentar carregar o primeiro programa completo encontrado
        program_dir = saved_full_programs[0]
        model_name = program_dir.stem.replace("_full", "")
        
        print(f"🔄 Carregando programa completo: {model_name}")
        print(f"   Diretório: {program_dir}")
        
        try:
            loaded_test_agent = dspy.load(str(program_dir))
            print("✅ Modelo carregado com sucesso!")
            
            print("\n🧪 Testando modelo carregado...")
            print("-"*60)
            
            # Teste rápido
            test_request = "What flights are available from SFO to JFK?"
            try:
                test_result = loaded_test_agent(user_request=test_request)
                print(f"Request: {test_request}")
                print(f"\nResponse: {test_result.response[:200]}...")
                print("\n✅ Modelo carregado funciona corretamente!")
            except Exception as e:
                print(f"❌ Erro ao testar modelo: {e}")
                import traceback
                traceback.print_exc()
        
        except Exception as e:
            print(f"❌ Erro ao carregar modelo: {e}")
            import traceback
            traceback.print_exc()
    
    else:
        print("ℹ️ Nenhum programa completo salvo encontrado para teste.")
        print("   Execute a serialização com save_program=True primeiro.")
        print("\n💡 Alternativamente, você pode testar carregando apenas o estado:")
        print("   # Reconstruir estrutura")
        print("   test_agent = dspy.ReAct(DSPyAirlineCustomerService, tools=tools, max_iters=10)")
        print("   # Carregar estado")
        print("   test_agent.load('saved_models/agent_bootstrap_optimized_state.json')")
else:
    print("ℹ️ Execute a otimização e serialização primeiro.")


🧪 Testando Modelo Carregado (se disponível)
🔄 Carregando programa completo: agent_bootstrap_optimized
   Diretório: saved_models/agent_bootstrap_optimized_full
✅ Modelo carregado com sucesso!

🧪 Testando modelo carregado...
------------------------------------------------------------
Request: What flights are available from SFO to JFK?

Response: Hello,

I’ve checked the flight availability from **San Francisco (SFO) to New York (JFK)** for the next two days:

| Flight | Departure | Arrival | Price (USD) | Seats Available |
|--------|---------...

✅ Modelo carregado funciona corretamente!


## 📊 Análise dos Resultados de Otimização

### 🎯 O que Esperar da Otimização

**Melhorias Típicas**:
- ✅ **Task Completion**: +10-20% em tarefas complexas
- ✅ **Tool Accuracy**: +15-25% em seleção de ferramentas
- ✅ **Response Quality**: +10-15% em qualidade de respostas
- ✅ **Efficiency**: +5-10% em uso eficiente de ferramentas

**Limitações**:
- ⚠️ Performance depende do modelo base (LLM)
- ⚠️ Dataset pequeno pode limitar melhorias
- ⚠️ Alguns casos edge podem permanecer desafiadores
- ⚠️ Otimização leva tempo (investimento único)

### 💡 Por que Otimização é Crítica para Agentes

1. **🎯 Multi-Step Reasoning**: Agentes precisam fazer múltiplas decisões sequenciais
2. **🔧 Tool Selection**: Escolher ferramentas erradas causa falhas em cascata
3. **💬 Context Awareness**: Respostas devem considerar contexto completo
4. **⚡ Efficiency**: Menos iterações = melhor experiência do usuário

### 🔄 Próximos Passos

Após otimização inicial:
1. **📊 Monitorar em Produção**: Acompanhar métricas reais
2. **🔄 Iterar**: Adicionar exemplos baseado em erros comuns
3. **🎯 Refinar Métricas**: Ajustar pesos baseado em prioridades
4. **🤝 Ensemble**: Considerar combinar múltiplos agentes otimizados


In [103]:
# 🎯 Testando Agente Otimizado em Cenários Reais

print("\n🎯 Testando Agente Otimizado")
print("="*60)

# Usar o melhor agente disponível
if 'optimized_agent_mipro' in locals():
    best_agent = optimized_agent_mipro
    agent_name = "MIPRO Optimized"
elif 'optimized_agent_bootstrap' in locals():
    best_agent = optimized_agent_bootstrap
    agent_name = "BootstrapFewShot Optimized"
else:
    best_agent = agent
    agent_name = "Baseline"

print(f"Usando: {agent_name}\n")

# Teste 1: Cenário complexo
test_case_1 = "I'm Adam. I need to book the cheapest flight from SFO to JFK on 2025-09-15, and then I want to see my profile."
print("📝 Teste 1: Tarefa Complexa Multi-Step")
print(f"   Request: '{test_case_1}'")
print("\n" + "-"*60)
result_1 = best_agent(user_request=test_case_1)
print(result_1.response)
print("\n" + "="*60 + "\n")

# Teste 2: Query simples
test_case_2 = "What flights go from JFK to LAX?"
print("📝 Teste 2: Query Simples")
print(f"   Request: '{test_case_2}'")
print("\n" + "-"*60)
result_2 = best_agent(user_request=test_case_2)
print(result_2.response)
print("\n" + "="*60 + "\n")

# Teste 3: Suporte
test_case_3 = "I need help with a delayed baggage issue. My name is Sarah."
print("📝 Teste 3: Suporte")
print(f"   Request: '{test_case_3}'")
print("\n" + "-"*60)
result_3 = best_agent(user_request=test_case_3)
print(result_3.response)
print("\n" + "="*60)



🎯 Testando Agente Otimizado
Usando: BootstrapFewShot Optimized

📝 Teste 1: Tarefa Complexa Multi-Step
   Request: 'I'm Adam. I need to book the cheapest flight from SFO to JFK on 2025-09-15, and then I want to see my profile.'

------------------------------------------------------------
Hello Adam,

**Flight availability for 2025‑09‑15 (SFO → JFK)**  
- **AA101** – $450.00 – *0 seats available*  
- **UA205** – $380.00 – *0 seats available*  

Unfortunately, both flights are fully booked for the date you requested, so I’m unable to secure a reservation at this time.

**Your profile information**

| Field                     | Value                     |
|---------------------------|---------------------------|
| **Name**                  | Adam |
| **User ID**               | user_001 |
| **Email**                 | adam@example.com |
| **Phone**                 | +1‑555‑0101 |
| **Frequent Flyer Number**| FF12345 |

**What you can do next**

1. **Check alternative dates** – I can loo

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python

Hello Sarah,

I’ve filed a support ticket for your delayed baggage issue. Your ticket ID is **15bc4d7e-6e9e-4f68-9f7c-e4a5f8cd5c30**. Our baggage team will investigate the situation and contact you shortly with an update.

If you have any additional information (e.g., a description of the bag, flight details, or a preferred contact method), please let me know so we can expedite the resolution. You can also reference this ticket ID in any future communications with us.

Thank you for bringing this to our attention, and we apologize for the inconvenience.



## Example Usage (Antes da Otimização)

Let's test our customer service agent with various requests.

In [104]:
# Example 1: Book a flight
user_request = "Please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam"

print("User Request:", user_request)
print("\n" + "="*50 + "\n")

result = agent(user_request=user_request)
print("Agent Response:")
print(result.response)

User Request: Please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam


Agent Response:
Hello Adam,

I’ve checked the flight inventory for September 1, 2025, from San Francisco (SFO) to New York (JFK). Unfortunately, both flights we have on that day (American Airlines AA101 departing at 08:00 and United UA205 departing at 14:00) are fully booked, with no seats available at the moment.

Here are a few options you might consider:

1. **Select a different travel date** – I can look up availability for dates before or after September 1.
2. **Try a nearby airport** – For example, flying out of Oakland (OAK) or into Newark (EWR) may have seats.
3. **Join a waiting list** – I can place you on a waitlist for one of the flights, and we’ll notify you if a seat opens up.
4. **Explore alternative airlines or routes** – I can search for connecting flights or other carriers that might have availability.

Please let me know which option you’d like to pursue, or if there’s anything

In [105]:
# Example 2: Check available flights
user_request2 = "What flights are available from JFK to LAX?"

print("User Request:", user_request2)
print("\n" + "="*50 + "\n")

result2 = agent(user_request=user_request2)
print("Agent Response:")
print(result2.response)

User Request: What flights are available from JFK to LAX?


Agent Response:
Here are the flights we have available from **JFK** to **LAX** for **November 1, 2025**:

| Flight | Airline | Departure | Arrival | Duration | Price (USD) | Seats Available |
|--------|---------|-----------|---------|----------|-------------|-----------------|
| **DL302** | Delta Air Lines | 10:00 AM (JFK) | 1:30 PM (LAX) | 6 h 30 m | $520.00 | 12 |

If you’d like to book this flight, see options for other dates, or need any additional assistance, just let me know!


In [106]:
# Example 3: Get user information
user_request3 = "Can you look up my profile? My name is Sarah"

print("User Request:", user_request3)
print("\n" + "="*50 + "\n")

result3 = agent(user_request=user_request3)
print("Agent Response:")
print(result3.response)

User Request: Can you look up my profile? My name is Sarah


Agent Response:
Sure, Sarah! Here are the details we have on file for you:

- **Name:** Sarah  
- **User ID:** user_002  
- **Email:** sarah@example.com  
- **Phone:** +1‑555‑0102  
- **Frequent Flyer Number:** (not set)

If any of this information needs to be updated or if you have any other requests, just let me know!


In [107]:
# Example 4: File a support ticket
user_request4 = "I need to file a complaint about delayed baggage. My name is Adam."

print("User Request:", user_request4)
print("\n" + "="*50 + "\n")

result4 = agent(user_request=user_request4)
print("Agent Response:")
print(result4.response)

User Request: I need to file a complaint about delayed baggage. My name is Adam.


Agent Response:
Hello Adam,

Your complaint about delayed baggage has been successfully filed. Your support ticket ID is **4a0fbfa0-def4-4de8-893a-447d94523c9c**. 

We’re sorry for the inconvenience you’ve experienced. If you have any additional details to add, need updates on the status of your baggage, or require further assistance, please let us know.

Thank you for bringing this to our attention.


## Inspect Database State

Let's check what happened in our mock databases after the agent interactions.

In [ ]:
print("Itineraries Database:")
for itinerary_id, itinerary in itineraries_db.items():
    print(f"ID: {itinerary_id}")
    print(f"User: {itinerary.user_id}")
    print(f"Status: {itinerary.status}")
    print(f"Total Price: ${itinerary.total_price}")
    print(f"Flights: {len(itinerary.flights)}")
    for flight in itinerary.flights:
        print(f"  - {flight.flight_number}: {flight.departure_airport} → {flight.arrival_airport}")
    print("\n")

print("\nTickets Database:")
for ticket_id, ticket in tickets_db.items():
    print(f"ID: {ticket_id}")
    print(f"User: {ticket.user_id}")
    print(f"Status: {ticket.status}")
    print(f"Issue: {ticket.issue_description}")
    print("\n")

Itineraries Database:
ID: cce6d281-b79d-43e9-8e9a-2e9e53270095
User: user_001
Status: confirmed
Total Price: $450.0
Flights: 1
  - AA101: SFO → JFK


ID: 85e966ea-bc93-4019-9a33-4224db5c4e7f
User: user_001
Status: confirmed
Total Price: $380.0
Flights: 1
  - UA205: SFO → JFK


ID: bb62c0c7-0abb-4e63-a20e-68b751c2058b
User: user_001
Status: confirmed
Total Price: $380.0
Flights: 1
  - UA205: SFO → JFK


ID: 3d6d718d-4be8-44c0-9a32-c9be936d6de2
User: user_001
Status: confirmed
Total Price: $450.0
Flights: 1
  - AA101: SFO → JFK


ID: f2b4c159-8b34-40fc-a520-afaccf14a70b
User: user_001
Status: confirmed
Total Price: $380.0
Flights: 1
  - UA205: SFO → JFK


ID: e523097d-ff4e-4c15-9e9a-bc86330a396d
User: user_001
Status: confirmed
Total Price: $380.0
Flights: 1
  - UA205: SFO → JFK


ID: a4c19e4d-74c6-4709-bd2e-6b9df58e9443
User: user_001
Status: confirmed
Total Price: $380.0
Flights: 1
  - UA205: SFO → JFK


ID: b49a8f7c-596d-4743-971f-440c221333a4
User: user_001
Status: confirmed
Total Pr

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/Documents/Work/MyProjects/ai_materials/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/Users/joaogabriellima/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Users/joaogabriellima/.local/share/uv/python

## Summary

This notebook demonstrates how to build and **optimize** a sophisticated customer service agent using DSPy's ReAct module. The agent can:

1. **Understand user requests** in natural language
2. **Reason about** what tools to use
3. **Execute tool calls** to fetch information, book flights, and manage itineraries
4. **Provide helpful responses** to users

### Key Components

- **Data Models**: Pydantic models for structured data
- **Tools**: Functions that interact with airline systems
- **Agent**: DSPy ReAct agent that orchestrates tool usage
- **LLM Integration**: Uses Groq/OpenAI models for reasoning
- **Optimization**: Advanced techniques (MIPRO, BootstrapFewShot) to improve performance

### Advanced Optimization Features

- **Multi-Objective Metrics**: Optimizes for task completion, tool accuracy, response quality, and efficiency
- **MIPRO Optimization**: Advanced instruction and example optimization
- **Few-Shot Learning**: Dataset-driven optimization for better generalization
- **Performance Evaluation**: Comprehensive metrics to measure agent effectiveness

### Why Optimization Matters for Agents

The ReAct architecture allows the agent to alternate between reasoning and acting, but **optimization is crucial** because:

1. **Tool Selection**: Wrong tool choices cause cascading failures
2. **Multi-Step Reasoning**: Agents make sequential decisions that compound errors
3. **Context Awareness**: Responses must consider full conversation context
4. **Efficiency**: Fewer iterations = better user experience

### Results

After optimization with MIPRO/BootstrapFewShot, you should see improvements in:
- ✅ Task completion rates (+10-20%)
- ✅ Tool usage accuracy (+15-25%)
- ✅ Response quality (+10-15%)
- ✅ Operational efficiency (+5-10%)

The optimized agent demonstrates how DSPy's optimization techniques can significantly improve agentic workflows, making them more reliable, efficient, and effective for production use.